# IMU Data Classification

In [1]:
# Visualisation
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
import dataframe_image as dfi
import warnings

# Statistics
import pandas as pd
import numpy as np

# Data processing
from sklearn.impute import KNNImputer

# Machine learning/Deep learning
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
from keras.callbacks import TensorBoard, ReduceLROnPlateau

# Helper functions
from helper.helper_filter import *
from helper.helper_preprocess import *
from helper.helper_train import *

## Filter data
### Extract data tables and visualise

In [2]:
# Read the raw data from each target action and store them in a list
lqw_raw = load_data("./IMU_Data/LGW")
ramp_ascend_raw = load_data("./IMU_Data/Ramp_ascend")
ramp_descend_raw = load_data("./IMU_Data/Ramp_descend")
sit_to_stand_raw = load_data("./IMU_Data/Sit_to_stand")
stand_to_sit_raw = load_data("./IMU_Data/Stand_to_sit")

folders = [lqw_raw, ramp_ascend_raw, ramp_descend_raw, sit_to_stand_raw, stand_to_sit_raw]

In [ ]:
# Preview some of the data to check format
lqw_raw[0].data.describe()

In [4]:
lqw_raw[0].data.head()
dfi.export(lqw_raw[0].data, "assets/table_1.png", max_rows=10, max_cols=6)

In [ ]:
# Plot histograms to visualize all data
lqw_raw[0].data.hist(bins=50,figsize=(30,30))

In [ ]:
# Check entries that are outside of the standard deviation
std_table = []
head = ["Action","File name", "Column name", "Mean", "Standard deviation", "#entries>5std", "#entries<5std"]

for folder in folders:
    for file in folder:
        for column in file.data:
            mean = file.data[column].mean()
            std = file.data[column].std()
            count_above_std5 = 0
            count_below_std5 = 0

            for entry in file.data[column]:
                if entry < mean - std*5:
                    count_below_std5 += 1
                elif entry > mean + std*5:
                    count_above_std5 += 1

            if count_above_std5 > 0 or count_below_std5 > 0:
                std_table.append([file.folder_name, file.file_name, column, format(mean, '.4f'), format(std, '.4f'), count_above_std5, count_below_std5]) # add data for every column

print(tabulate(std_table, headers=head, tablefmt="grid"))

### Check for unwanted columns
From the table above, we can see how multiple timestamps have been used across different files. It was decided to investigate further whether the timestamps are aligned and can be ignored. It can be seen how, under the LGW some files are missing "Sync" and "Offset" timestamp, so it was decided to remove all columns that contains them to ensure consistency across the data. Additionally, the LWR from SV misses the timestamp from the Right sensors and Thigh. The timestamps that appear across al columns are 'Shank_L_Timestamp', 'Foot_L_Timestamp', 'Pelvis_Timestamp', arguebly one of them should be used as the baseline time.

The total number of entries is plotted as well, it can be seen how the majority of the data comes from the ground walking action and less from the standing and sitting actions. This might result in a bias towards the former action mentioned.

In [ ]:
# Check number of columns in each dataframe
column_table = []
head = ["Action","File name", "Row Nr", "Column Nr", "Timestamp columns"]
df_table = pd.DataFrame(columns=["Action","File name", "Row Nr", "Column Nr", "Timestamp columns"])

for folder in folders:
    for file in folder:
        filtered_columns =[col for col in file.data.columns if "timestamp" in col.lower()]
        column_table.append([file.folder_name, file.file_name, file.data.index.size, len(file.data.columns), filtered_columns])
        df_table.loc[len(df_table)] = [file.folder_name, file.file_name, file.data.index.size, len(file.data.columns), filtered_columns]
        style = df_table.style.set_properties(subset=['Timestamp columns'], **{'width': '500px'})

table = tabulate(column_table, headers=head, tablefmt='grid')

print(table)

In [8]:
dfi.export(style, "assets/table_2.png")

In [ ]:
# Check if all timestamps columns have the same data inside a dataframe and check what is the difference in time between them
reference_columns = ['Shank_L_Timestamp', 'Foot_L_Timestamp', 'Pelvis_Timestamp']

for folder in folders:
    for file in folder[:2]:
        for ref in reference_columns:
            # Filter columns to get only those containing time
            time_columns = [col for col in file.data_filtered.columns if 'timestamp' in col.lower()]

            # Reference column for comparison
            ref_column = file.data_filtered[ref]
            time_difference = []
            for col in time_columns:
                time_difference.append(file.data_filtered[col] - ref_column)

            means = [sum(inner_array)/len(inner_array) for inner_array in time_difference]
            if max(means) > 1000.: # if difference is bigger than 1 seconds
                print(f"Using {ref} - Different timestamp in {file.file_name} with maximum value: {format(max(means), '.3f')}")
                pass

In [10]:
# Drop all columns that contain sync, annotations and offset timestamps
for folder in folders:
    for file in folder:
        # Drop all columns that contain sync, annotations and offset timestamps
        file.data_filtered.drop(columns=[col for col in file.data_filtered.columns if 
                                any(info in col.lower() for info in ["sync", "offset", "annotation"])], inplace=True)
        
        # Drop all timestamp columns that are not "Shank_L_Timestamp"
        for column in file.data_filtered.columns:
            if "timestamp" in column.lower():
                if column.lower() != "shank_l_timestamp":
                    file.data_filtered.drop(columns=column, inplace=True)
        
        # Replace column name and place as the first index 
        file.data_filtered.rename(columns={'Shank_L_Timestamp': 'Timestamp'}, inplace=True)
        col = file.data_filtered.pop('Timestamp')
        file.data_filtered.insert(0, col.name, col)

In [ ]:
# Check number of columns in each dataframe
column_table = []
head = ["Action","File name", "Row Nr", "Column Nr"]

for folder in folders:
    for file in folder:
        column_table.append([file.folder_name, file.file_name, file.data.index.size, len(file.data.columns)])

print(tabulate(column_table, headers=head, tablefmt='grid'))

### Check for NaNs
It can be observed how the only files that contains NaNs are normal_walk_lg_trial_01.dat and normal_walk_lg_trial_02.dat. Both files contain 1521 entry with 17 or 56 NaN entries in individual columns. The NaN values constitute 1.12% and 3.68%, respectively of the total entries. A nearest neighbors imputation strategy is used to replace the missing data from the set. Originally, a simple imputation was used with a "median" strategy, but, after checking the data, all of the features that need imputation are Gaussian distributed (except the Pelvic magnometometer data that has two peaks). It is better to replace the missing data with a Gaussian distributed set of values compared to a constant. k-Nearest Neighbors offers the advantage of tuning the missing values by using the neighboring entries. 

In [ ]:
# Check number of columns in each dataframe
nan_table = []
head = ["Action","File name", "NaN total number", "NaN columns"]
columns_to_visualize = []
df_table = pd.DataFrame(columns=["Action","File name", "NaN total number", "NaN columns"])

for folder in folders:
    for file in folder:
        nan_number = file.data_filtered.isnull().sum().sum()
        
        # Add to table only if there are NaN values
        if nan_number > 0:
            nan_columns = ""
            nan = ""
            columns_to_visualize.append(file.data_filtered)
            
            # Check which columns have NaN values and how many
            for col in file.data_filtered.columns:
                if file.data_filtered[col].isnull().sum() > 0:
                    nan_columns += col + "=" + str(file.data_filtered[col].isnull().sum()) + "\n"
                    nan += col + "=" + str(file.data_filtered[col].isnull().sum()) + ", "
            
            nan_table.append([file.folder_name, file.file_name, nan_number, nan_columns])
            df_table.loc[len(df_table)] = [file.folder_name, file.file_name, nan_number, nan]

print(tabulate(nan_table, headers=head, tablefmt='grid'))

style = df_table.style.set_properties(subset=['NaN columns'], **{'width': '500px'})
dfi.export(style, "assets/table_3.png")

In [ ]:
# Plot histograms to visualize all data
for visualize in columns_to_visualize:
    visualize.hist(bins=50,figsize=(30,30))

In [14]:
# Replace NaN values with the k-Nearest Neighbor
for folder in folders:
    for file in folder:
        if file.data_filtered.isnull().sum().sum() > 0:
            imputer = KNNImputer(n_neighbors=5)
            file.data_filtered = pd.DataFrame(imputer.fit_transform(file.data_filtered),columns = file.data_filtered.columns)

In [15]:
# Check if any NaN values are left
for folder in folders:
    for file in folder:
        if file.data_filtered.isnull().sum().sum() > 0: 
            print("NaN values left")

## Preprocess data
### Apply filtering
Check IMU data against vibrations; accelerometer may record high frequency noise due to vibration. It was calculated that the sampling time is about 9.8 miliseconds, frequency of about 102.4Hz.

In [ ]:
# Check the filter and find appropriate parameters
data = folder[0].data_filtered
old_data = folder[0].data["Thigh_R_Gyroscope_Y"]
new_data = None

# Sampling frequency
ts = data["Timestamp"].diff().median()  # Sampling time
fs = 1000/ts                            # Sampling frequency

# Filter design
N = 3               # Order of the filter 
cutoff = 25.0       # Cutoff frequency

# Apply filter
new_data = low_pass_filter(ts, old_data)

# Plot the original and filtered signals
plt.figure(figsize=(15, 8))
start = 0
end = 500

plt.subplot(2, 1, 1)
plt.plot(data["Timestamp"][start:end], old_data[start:end])
plt.title('Original Signal', fontsize=20)
plt.xlabel('Time [s]', fontsize=16)
plt.ylabel('Amplitude', fontsize=16)

plt.subplot(2, 1, 2)
plt.plot(data["Timestamp"][start:end], new_data[start:end])
plt.title('Filtered Signal (Low-pass Filter)', fontsize=20)
plt.xlabel('Time [s]', fontsize=16)
plt.ylabel('Amplitude' , fontsize=16)

plt.tight_layout()
plt.show()

In [17]:
# Remove outliers and smooth curve using a low pass filter
for folder in folders:
    for file in folder:
        # Extract sampling time
        ts = file.data_filtered["Timestamp"].diff().median() # Median sampling time

        # Remove outliers
        for name, data in file.data_filtered.items():
            if name != 'Timestamp':
                data = low_pass_filter(ts, data)

### Apply the slinding window technique

In [ ]:
# Check if technique works
tw = 350    # window size
dt = 50     # time increment

test = folder[0].data_filtered
output = pd.DataFrame()

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
    output = generate_features(test, tw, dt)

display(output)

In [19]:
tw = 350        # window size
dt = 50         # window step

# Apply the moving average filter to the data and get all features
# Takes around 30 minutes to run
for folder in folders:
    for file in folder:
        # Apply the slinding window to the data
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
            file.data_processed = generate_features(file.data_filtered, tw, dt)

        # Drop first row where the gradient is 0
        file.data_processed = file.data_processed.iloc[1:]

In [ ]:
# Check if technique works
dfi.export(file.data_processed, "assets/table_4.png", max_rows=10, max_cols=5)
file.data_processed.head()

In [21]:
def get_one_hot_encoding(folders):
    names = []
    for folder in folders:
        names.append(folder[0].folder_name)
    return pd.get_dummies(names)

In [ ]:
names_one_hot = get_one_hot_encoding(folders)
dfi.export(names_one_hot, "assets/table_5.png")
names_one_hot.head()

In [23]:
# Combine all five actions into one dataframe and set the target labels 
iterator = 1
all_df = []

for folder in folders:    
    # Create single dataframe for action
    df = pd.DataFrame()
    df = pd.concat([file.data_processed for file in folder])

    # Add target labels
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
        df["Action"] = iterator
        iterator = iterator + 1
    
    # Add dataframe to the list
    all_df.append(df)

# Combine all dataframes into one
df = pd.concat(all_df)

In [ ]:
all_df[3]

## Train models

Start by splitting the data into training and testing.

In [25]:
# Get csv data
df = pd.read_csv('combined_data.csv')

In [26]:
# Split data
X = df.iloc[:, :-1]     # Input features
y = df.iloc[:, -1:]     # Target labels

# Split data into training (70%) and testing set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)

In [ ]:
print("The dimension of train is {}".format(X_train.shape))
print("The dimension of test is {}".format(X_test.shape))

In [ ]:
value_count = pd.DataFrame()
data_train = y_train['Action'].value_counts()
data_test = y_test['Action'].value_counts()

value_count = pd.concat([data_train, data_test], axis=1)
value_count.insert(0, "Action", ['LGW', ' Ramp_ascend', 'Ramp_descend', 'Sit_to_stand', 'Stand_to_sit'], True)
value_count = value_count.set_axis(['Action', 'Train Value Count', 'Test Value Count'], axis=1)
value_count

In [29]:
dfi.export(value_count, "assets/table_6.png")

### ANN

In [ ]:
ann = ANN(X_train, X_test, y_train, y_test)
ann.run_pipeline()
ann.evaluate()

In [ ]:
ann.best_params

### SVM

In [ ]:
svm = SVM(X_train, X_test, y_train, y_test)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    svm.run_pipeline()
svm.evaluate()

In [ ]:
svm.best_params

### CNN

In [ ]:
# Combine all five actions without any features extraction 
# into one dataframe and set the target labels
iterator = 1
all_df = []

for folder in folders:    
    # Create single dataframe for action
    df_cnn = pd.DataFrame()
    df_cnn = pd.concat([file.data_filtered for file in folder])

    # Add target labels
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
        df_cnn["Action"] = iterator
        iterator = iterator + 1
    
    # Add dataframe to the list
    all_df.append(df_cnn)

# Combine all dataframes into one
df_cnn = pd.concat(all_df)
df_cnn.drop(columns=['Timestamp'], inplace=True)

df_cnn.head()

In [35]:
# Split data CNN
X_cnn = df_cnn.iloc[:, :-1]     # Input features
y_cnn = df_cnn.iloc[:, -1:]     # Target labels

# Split data into training (70%), validation (15%) and testing set (15%)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.3, random_state=109)
X_val_cnn, X_test_cnn, y_val_cnn, y_test_cnn = train_test_split(X_test_cnn, y_test_cnn, test_size=0.5, random_state=109)

### ResNet19

Generate a ResNet model inspired by: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9071921/ and adapt it for IMU data. Originally the network had 50 layers and was created for Electrocardiogram (ecg) automated diagnosis. In this scenario, the network was decreased to 19 layers (ResNet19) and the residuals layers were kept to 441 (take all sensor's data). Despite the succesfull formulation, the network is overparameterised, with multiple residual blocks of weights close to 0. To save on computation power, storage for the network and potential overfitting, a simple network was defined below.

In [ ]:
# ResNet19: Define dimensions
nclasses = 6
input_dims = (63, 1)  # Input dimensions
residual_blocks_dims = [[63, 32],   # Output of conv layer       / Input of 1st residual blk
                        [63, 128],  # Output of 1st residual blk / Input of 2st residual blk
                        [63, 256]]  # Output of 2th residual blk / Input of dense layer

# Get model
model_resnet = ResNet1D(input_dims, residual_blocks_dims, nclasses)

# Present model summary
svg = plot_model(model_resnet, to_file='./assets/cnn_model_resnet.png', show_shapes=True, show_layer_names=True)
model_resnet.summary()

In [ ]:
# Optimization parameters
lr = 0.1            # Learning rate
epochs = 15         # Number of epochs
batch_size = 64     # Batch size
callbacks = []

# Reduce learning rate on platteu; reduce the learning rate 
# when the validation loss stops improving. Addopted from
# https://arxiv.org/abs/1711.05225.
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=10, min_lr=lr/10000))

# Create file for tensorboard visualization, run:
# $ tensorboard --logdir=./
callbacks.append(TensorBoard(log_dir='./logs', write_graph=False))

# Compile model
model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

# Train model
history_resnet = model_resnet.fit(X_train_cnn, y_train_cnn,
                batch_size=batch_size, 
                epochs=epochs,
                validation_data=(X_val_cnn, y_val_cnn), 
                callbacks=callbacks)

In [ ]:
loss, accuracy = model_resnet.evaluate(X_test_cnn, y_test_cnn, batch_size=64, verbose=1)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
# Summarize history for accuracy
plt.plot(history_resnet.history['accuracy'])
plt.plot(history_resnet.history['val_accuracy'])
plt.title('ResNet1D CNN Model Accuracy', fontsize=20)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.legend(['train', 'validation'])
plt.show()

# Summarize history for loss
plt.plot(history_resnet.history['loss'])
plt.plot(history_resnet.history['val_loss'])
plt.title('ResNet1D CNN Model Loss', fontsize=20)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.legend(['train', 'validation'])
plt.show()

### Simple Network

In [ ]:
# Simple model
lr = 0.1                # Learning rate
epochs = 10             # Number of epochs
batch_size = 64         # Batch size
nclasses = 6            # len(Output classes) + 1
input_dims = (63, 1)   # Input dimensions
callbacks = []

# Reduce learning rate on platteu; reduce the learning rate 
# when the validation loss stops improving. Addopted from
# https://arxiv.org/abs/1711.05225.
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=10, min_lr=lr/10000))

# Create file for tensorboard visualization, run:
# $ tensorboard --logdir=./
callbacks.append(TensorBoard(log_dir='./logs', write_graph=False))

# Get model
model_simple = CNN(input_dims, nclasses)
model_simple.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = model_simple.fit(X_train_cnn, y_train_cnn, 
                 epochs=10, 
                 batch_size=32, 
                 validation_data=(X_val_cnn, y_val_cnn),
                 callbacks=callbacks)

In [ ]:
svg = plot_model(model_simple, to_file='./assets/cnn_model_simple.png', show_shapes=True, show_layer_names=True)
model_simple.summary()

In [ ]:
# Evaluate simple model
loss, accuracy = model_simple.evaluate(X_test_cnn, y_test_cnn, batch_size=32, verbose=1)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Simple CNN Model Accuracy', fontsize=20)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.legend(['train', 'validation'])
plt.show()

# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Simple CNN Model Loss', fontsize=20)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
y_pred_cnn = model_simple.predict(X_test_cnn)

# Convert probabilities to class labels
predicted_classes_cnn = np.argmax(y_pred_cnn, axis=1)

# Plot Confusion Matrix
labels = [label_map[i] for i in ann.model.classes_]
plot_cm(y_test_cnn, predicted_classes_cnn, labels)

### Comparison

Find the 15 most relevant weights for both ANN and SVG. 

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ann_features, ann_scores = ann.get_most_relevant_features(X_train.columns,
                                                                X_train.values,
                                                                y_train.values)

# Plot
plt.barh(ann_features, ann_scores)
plt.xlabel("Permutation Importance")

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    svm_features, svm_scores = svm.get_most_relevant_features(X_train.columns,
                                                                X_train.values,
                                                                y_train.values)

# Plot 
plt.barh(svm_features, svm_scores)
plt.xlabel("Permutation Importance")


In [ ]:
# Look at correlation between high coefficinet features
substrings = ["Gyroscope", "Action"]
cols_to_keep = [col for col in df.columns if 'Gyroscope_X_max' in col or 'Action' in col]
corr_df = df[cols_to_keep]
heatmap = sns.heatmap(corr_df.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

In [ ]:
# Look at correlation between high coefficinet features
plt.figure(figsize=(16, 6))
substrings = ["Gyroscope", "Action"]
cols_to_keep = [col for col in df.columns if 'Gyroscope_X_min' in col or 'Action' in col or 'Gyroscope_Y_min' in col]
corr_df = df[cols_to_keep]
heatmap = sns.heatmap(corr_df.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

Find the most significant segment which contributes to the activities recognition based on the ANN prediction

In [49]:
# Separate data based on the segment
sensors = ["Foot_L", "Foot_R", "Thigh_L", "Thigh_R", "Shank_L", "Shank_L", "Pelvis"]
sensors_data = {}

# Get all columns that contain the sensor name
for sensor in sensors:
    columns = [col for col in df.columns if (sensor in col or "Action" in col)]
    sensors_data[sensor] = df[columns]

In [ ]:
# Print first sensor data
first_entry = next(iter(sensors_data.values()))
first_entry.head()

In [ ]:
models = []

# Get the accuracy of the ANN on each separate data
for key, data in sensors_data.items():
    # Split data
    X = data.iloc[:, :-1]     # Input features
    y = data.iloc[:, -1:]     # Target labels

    # Split data into training (70%) and testing set (30%)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.7, random_state=42)

    # Create network and get accuracy
    sensor_ann = ANN(X_tr, X_te, y_tr, y_te)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sensor_ann.run_pipeline()
        print(f'{key} accuracy : {sensor_ann.get_accuracy()}')

    # Add model to the list
    models.append(ModelToCompare(key , sensor_ann.get_accuracy(), sensor_ann, X_tr, X_te, y_tr, y_te))

In [ ]:
# Check highest accuacy
max_acc_model = max(models, key=lambda item: item.accuracy) 
max_acc_model.model.evaluate()

In [ ]:
# Using only the most relevant feature, calculate the clasification error using ANN
print(f"Classification error of the ANN using {max_acc_model.sensor}: {max_acc_model.model.get_classification_error()}")

In [54]:
sensor = sensors_data[max_acc_model.sensor]

In [ ]:
sensor.head()

In [ ]:
# Using only the most relevant feature, calculate the clasification error using SVM
sensor = sensors_data[max_acc_model.sensor]

# Split data
X = sensor.iloc[:, :-1]     # Input features
y = sensor.iloc[:, -1:]     # Target labels

# Split data into training (70%) and testing set (30%)
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.7, random_state=42)

max_svm_model = SVM(X_tr, X_te, y_tr, y_te)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    max_svm_model.run_pipeline()
print(f"Accuracy of the SVM using {max_acc_model.sensor}: {max_svm_model.get_accuracy()}")
print(f"Classification error of the SVM using {max_acc_model.sensor}: {max_svm_model.get_classification_error()}")
max_svm_model.evaluate()